## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-17-20-38-58 +0000,nypost,Bill de Blasio throws jilted ex-gal pal Nomiki...,https://nypost.com/2025/11/17/us-news/bill-de-...
1,2025-11-17-20-35-29 +0000,nypost,Ezra Klein’s private briefing with Senate Dems...,https://nypost.com/2025/11/17/media/ezra-klein...
2,2025-11-17-20-35-00 +0000,wsj,Fed Governor Lisa Cook Offers First Defense of...,https://www.wsj.com/politics/policy/fed-govern...
3,2025-11-17-20-32-00 +0000,wsj,New York Officials to Team Up With Wall Street...,https://www.wsj.com/us-news/new-york-officials...
4,2025-11-17-20-27-00 +0000,wsj,Poland Says Rail Explosion Was ‘Unprecedented ...,https://www.wsj.com/world/europe/poland-says-r...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2392/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
155,trump,33
29,new,14
158,epstein,11
43,up,9
258,files,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
224,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...,91
186,2025-11-17-10-21-07 +0000,nyt,Trump Says House Republicans Should Vote to Re...,https://www.nytimes.com/2025/11/16/us/politics...,75
60,2025-11-17-18-21-31 +0000,nypost,Dozens arrested in NC as part of ‘Operation Ch...,https://nypost.com/2025/11/17/us-news/dozens-a...,73
115,2025-11-17-15-34-23 +0000,bbc,Trump calls on House Republicans to vote to re...,https://www.bbc.com/news/articles/cvgv653v1vjo...,73
74,2025-11-17-17-39-35 +0000,nypost,Trump-supporting grocer says he may shut down ...,https://nypost.com/2025/11/17/business/trump-s...,69


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
224,91,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...
50,54,2025-11-17-18-40-35 +0000,nypost,Bangladesh’s ousted PM Sheikh Hasina sentenced...,https://nypost.com/2025/11/17/world-news/bangl...
95,44,2025-11-17-16-57-00 +0000,wsj,Fed Vice Chair Philip Jefferson acknowledged t...,https://www.wsj.com/economy/central-banking/fe...
276,42,2025-11-16-20-56-21 +0000,nyt,"Britain’s Labour Government Plans New, Tougher...",https://www.nytimes.com/2025/11/16/world/europ...
74,36,2025-11-17-17-39-35 +0000,nypost,Trump-supporting grocer says he may shut down ...,https://nypost.com/2025/11/17/business/trump-s...
0,35,2025-11-17-20-38-58 +0000,nypost,Bill de Blasio throws jilted ex-gal pal Nomiki...,https://nypost.com/2025/11/17/us-news/bill-de-...
29,34,2025-11-17-19-37-44 +0000,nyt,France Agrees to Sell Up to 100 Rafale Fighter...,https://www.nytimes.com/2025/11/17/world/europ...
262,32,2025-11-16-23-33-06 +0000,nypost,Vile antisemitic graffiti scrawled on Mexico’s...,https://nypost.com/2025/11/16/world-news/antis...
156,32,2025-11-17-12-49-26 +0000,cbc,Dozens of religious pilgrims from India feared...,https://www.cbc.ca/news/world/saudi-medina-acc...
61,27,2025-11-17-18-20-00 +0000,wsj,Do These Five Things Now to Get the Most Out o...,https://www.wsj.com/personal-finance/premium-c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
